In [0]:

!pip install pmdarima #Installation of Auto-ARIMA


## is not always prudent to move immediately to the most advanced method for any given problem. The simpler models are often better, faster, and more interpretable.

In [0]:
import datetime
import math
from statsmodels.tsa.stattools import adfuller
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from cycler import cycler
import pandas as pd
import pmdarima as pm
import numpy as np
import matplotlib.pyplot as plt
import pmdarima as pm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns
from pandas.tseries.offsets import DateOffset
from sklearn import metrics


# Function to collecte Metrics 

def evaluate(y_true, y_pred):
    dict = { 'MAE':   metrics.mean_absolute_error(y_true, y_pred),
              'MSE':  metrics.mean_squared_error(y_true, y_pred),
              'RMSE': math.sqrt(metrics.mean_squared_error(y_true, y_pred)),
               'R2':  metrics.r2_score(y_true, y_pred)
            }
    return dict

In [0]:
pred_periods = 12

## Processing the Data

Data was extracted from **EDR** table named **FIN_INV_BR_BILLING**  from SAS  (EDR_CURATED_LAKE (CURATED)) using the query : 

```
SELECT DISTINCT t2.SNAPSHOT_PERIOD_END AS SNAPSHOT_PERIOD_END, \n
          t2.FY, 
          t2.POST_PERIOD, 
          /* SUM_of_BR_BILLED_AMT */
            (SUM(t2.BR_BILLED_AMT)) FORMAT=16.2 AS SUM_of_BR_BILLED_AMT
      FROM CURATED.FIN_INV_BR_BILLING t2
           INNER JOIN CURATED.DEMAND_BR_ITEMS t1 ON (t2.BR_NMBR = t1.BR_NMBR)
      WHERE t2.BR_BILLED_AMT >= 0
      GROUP BY t2.SNAPSHOT_PERIOD_END,
               t2.FY,
               t2.POST_PERIOD
      ORDER BY t2.FY,
               t2.POST_PERIOD;
```

**The data represents the positive revenue (Billed amount)aggregated per period for each FY for all clients/partners. **

The function below helps to read data from csv file and make the column SNAPSHOT_PERIOD_END as DateTimeindex, that way our forecasting analysis will be able to interpret these values. We drop the other columns as we do univariate analyses.

In [0]:
def upload_file(path):
    time_column_name = "Timestamp"
    df = pd.read_csv(path, delimiter=";")
    df['Timestamp'] = df['SNAPSHOT_PERIOD_END'].apply(lambda x: datetime.datetime.strptime(x, "%d%b%Y:%H:%M:%S").date())
    df.sort_values(by=['Timestamp'],inplace=True)
    df.set_index(df.Timestamp, inplace=True)
    df= df.drop(['SNAPSHOT_PERIOD_END','FY','COUNT_BR_NMBR','COUNT_GC_ORG','Timestamp'], axis=1)
    df= df.rename(columns={"SUM_of_BR_BILLED_AMT": "Revenue"})
    df['Revenue']=df['Revenue']/1000000
    return (df)

df= upload_file("./dataset2.csv")
df.head(100)

In [0]:
# Decompose the time series
plt.rcParams["figure.figsize"] = (8,6)
result = seasonal_decompose(df['Revenue'], model='multiplicative', period = 12)
result.plot()
plt.show()

<u>**NOTE :**</u>
**- A  12-month seasonal pattern is visible**
**- An upwards and downwards trend is evident**

In [0]:
def stationary_test(df,columns,treshold = 0.05):

    for quant in columns:
        
        # Remove null values (useful when plotting diff series)
        time_series = df[quant][df[quant].notnull()]

        # Autocorrelation function
        plot_acf(time_series,title='Autocorrelation function - {}'.format(quant))
        plt.xlabel('lags')
        plt.show()

        # Dickely-Fuller test statistics
        dickley_fuller_test = adfuller(time_series)
        pvalue = dickley_fuller_test[1]

        if pvalue < treshold:
            print('Time series "{}" is stationay (NULL HP rejected - pvalue = {:.4f})'.format(quant,pvalue))
        else:
            print('Time series "{}" might have unit root (NULL HP cannot be rejected - pvalue = {:.4f})'.format(quant,pvalue))
stationary_test(df, ['Revenue'])

# Make the time serie stationary

In [0]:
df_differenced = df.diff().dropna()
df_differenced.head()

## Stationarity in a time series indicates that a series’ statistical attributes, such as mean, variance, etc., are constant over time

In [0]:
stationary_test(df_differenced, ['Revenue'])

Differencing is a method of transforming a non-stationary time series into a stationary one. This is an important step in preparing data to be used in an ARIMA model.

## (1) Applying Auto-ARIMA/SARIMA Model (Autoregressive Integrated Moving Average)

Auto-ARIMA combines auto-regression, differencing, and a moving average into a single model using the following steps

Auto-ARIMA works by conducting differencing tests (i.e., Kwiatkowski–Phillips–Schmidt–Shin, Augmented Dickey-Fuller or Phillips–Perron) to determine the order of differencing, d, and then fitting models within ranges of defined start_p, max_p, start_q, max_q ranges. If the seasonal optional is enabled, auto-ARIMA also seeks to identify the optimal P and Q hyper- parameters after conducting the Canova-Hansen to determine the optimal order of seasonal differencing, D.

In [0]:
# split into train and test
split_number = df['Revenue'].count() - pred_periods # corresponds to a prediction horizion  06 months
df_train = pd.DataFrame(df['Revenue'][:split_number+1]).rename(columns={'Revenue':'y_train'})
df_test = pd.DataFrame(df['Revenue'][split_number:]).rename(columns={'Revenue':'y_test'})
# auto_arima
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
ax.plot(df_train)
ax.plot(df_test)

## Performing the Seasonal ARIMA
We noticed that our time series with a seasonal component, so it make sense to use a Seasonal ARIMA model. In order to do this we will need to choose p,d,q values for the ARIMA, and P,D,Q values for the Seasonal component.

In [0]:
# SARIMAX Model
model_revenue = pm.auto_arima(df_train, test='adf',                         
                          seasonal=True, m=12,
                          max_P=5, max_D=5, max_Q=5,trace=True,
                         error_action='ignore',  
                         suppress_warnings=True, 
                         stepwise=True)
# summarize the model characteristics
print(model_revenue.summary())

Note : For revenue forecasting, the resulting best model parameters gave us an AIC value of 567.164.  SARIMAX(4, 1, 0)x(0, 1, 0, 12)
The **Akaike information criterion** (AIC) is an estimator of the relative quality of statistical models for a given set of data. Given a collection of models for the data, AIC estimates the quality of each model, relative to each of the other models.

## Train the Model
We can then train the model by simply calling .fit on the stepwise model and passing in the training data:

In [0]:
model_revenue.fit(df_train)

In [0]:
# Generate prediction for n periods, 
# Predictions start from the last date of the training data
# test_pred = pd.DataFrame(model_fit.predict(n_periods=pred_periods),index=df_test.index )
# Forecast

prediction, confint = model_revenue.predict(n_periods=pred_periods, return_conf_int=True)
prediction_series = pd.Series(prediction,index=df_test.index)
cf= pd.DataFrame(confint)
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
ax.plot(df_train)
ax.plot(df_test)
ax.plot(prediction_series,color='Green')
ax.fill_between(prediction_series.index,
                cf[0],
                cf[1],color='grey',alpha=.3)

In [0]:
# collecting metrics

SARIMA_MAE = evaluate(df_test['y_test'], prediction_series)['MAE']
SARIMA_MSE = evaluate(df_test['y_test'], prediction_series)['MSE']
SARIMA_RMSE = evaluate(df_test['y_test'], prediction_series)['RMSE']
SARIMA_R2 = evaluate(df_test['y_test'], prediction_series)['R2']


In [0]:
# Generate a new time index from the last date on df
future_index=pd.date_range (start=df.last_valid_index(),periods=pred_periods, freq='M')

prediction, confint = model_revenue.predict(n_periods=pred_periods,  return_conf_int=True )
prediction_series = pd.Series(prediction,index=future_index)
cf= pd.DataFrame(confint)
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
ax.plot(df.Revenue)
ax.plot(prediction_series, color='green')
ax.fill_between(prediction_series.index,
                cf[0],
                cf[1],color='lightblue',alpha=.3)

prediction_series.head(12)

## (2) Applying Prophet

In [0]:
from fbprophet import Prophet,diagnostics
from sklearn.preprocessing import MinMaxScaler

df= upload_file("./Dataset2.csv")
# prepare expected column names
df_pro=pd.DataFrame(columns=['ds','y'])
df_pro=df.reset_index()
df_pro= df_pro.rename(columns={"Timestamp": "ds", "Revenue": "y"})
df_pro.head()


In [0]:

df_train = df_pro.iloc[:-pred_periods, :]
df_test = df_pro.iloc[int(-pred_periods):, :]
df_train.shape
df_test.shape


In [0]:
# set the uncertainty interval to 95% (the Prophet default is 80%)
m = Prophet(interval_width=0.95)
m.fit(df_train)


In [0]:
future = m.make_future_dataframe(periods=pred_periods,freq='M')
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(pred_periods)

In [0]:
from fbprophet.plot import plot_plotly
plot_plotly(m, forecast)

In [0]:
# Collecting metrics 

Prophet_MAE = evaluate(df_test['y'], forecast['yhat'].tail(pred_periods))['MAE']
Prophet_MSE = evaluate(df_test['y'], forecast['yhat'].tail(pred_periods))['MSE']
Prophet_RMSE = evaluate(df_test['y'], forecast['yhat'].tail(pred_periods))['RMSE']
Prophet_R2 = evaluate(df_test['y'], forecast['yhat'].tail(pred_periods))['R2']
evaluate(df_test['y'], forecast['yhat'].tail(pred_periods))

In [0]:
m = Prophet(interval_width=0.95)
m.fit(df_pro)
future = m.make_future_dataframe(periods=pred_periods,freq='M')
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(pred_periods)

In [0]:
from fbprophet.plot import plot_plotly
plot_plotly(m, forecast)

In [0]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(pred_periods)

## NeuralProphet

Installing Neural Prophet will crash your azure studio kernel

https://bobrupakroy.medium.com/neural-prophet-for-time-series-27bfeeaa5c7d

In [0]:
error_metrics = {'model':['SARIMA','Prophet'],
'MAE': [SARIMA_MAE,Prophet_MAE],
'MSE':[SARIMA_MSE,Prophet_MSE],
'RMSE':[SARIMA_RMSE,Prophet_RMSE],
'R2':[SARIMA_R2,Prophet_R2]}

In [0]:
acc_df = pd.DataFrame(error_metrics)
acc_df
fig, axes = plt.subplots(2,2,figsize=(16,8))

sns.lineplot(ax=axes[0,0], x='model',y='MAE', data=acc_df)
sns.lineplot(ax=axes[0,1],x='model',y='MSE', data=acc_df)
sns.lineplot(ax=axes[1,0],x='model',y='RMSE', data=acc_df)
sns.lineplot(ax=axes[1,1],x='model',y='R2', data=acc_df)

plt.show()
